In [159]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

| id | user_id | venue_id | latitude | longitude | created_at |
| --------- | --------- | ---------- | ------------------- |------------------- | --------------------- |
| 984301 | 2041916 | 5222 |  |  | 2012-04-21 17:39:01 |
| 984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47 |
| 984315 | 1764391 | 5222 |  |  | 2012-04-21 17:37:18 |
| 984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43 |
...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

﻿38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...﻿

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

﻿33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)﻿

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [25]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

checkins.dat удаляем последнюю строку: (1021966 rows)

In [121]:
data = pd.read_csv("umn_foursquare_datasets/checkins.dat", delimiter='|', skiprows=[1])

In [122]:
data.tail()

,id,user_id,venue_id,latitude,longitude,created_at
1021961,956377,845102,11195,42.7653662,-71.467566,2012-04-13 12:08:45
1021962,956119,1139114,29488,42.439479,-83.7438303,2012-04-13 11:36:44
1021963,956447,2088020,4432,,,2012-04-13 12:58:05
1021964,956733,960666,60,42.331427,-83.0457538,2012-04-13 21:56:19
1021965,957139,1771518,10935,,,2012-04-14 02:44:52


In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021966 entries, 0 to 1021965
Data columns (total 6 columns):
id            1021966 non-null int64
user_id       1021966 non-null int64
venue_id      1021966 non-null int64
latitude      1021966 non-null object
longitude     1021966 non-null object
created_at    1021966 non-null object
dtypes: int64(3), object(3)
memory usage: 46.8+ MB


In [124]:
data.rename(columns=lambda x: x.strip(), inplace=True)

In [132]:
data.created_at.iloc[1000]

' 2012-04-21 19:04:54'

In [135]:
data.select_dtypes(include=['object']).head()

,latitude,longitude,created_at
0,,,2012-04-21 17:39:01
1,38.8951118,-77.0363658,2012-04-21 17:43:47
2,,,2012-04-21 17:37:18
3,33.800745,-84.41052,2012-04-21 17:43:43
4,,,2012-04-21 17:42:58


In [137]:
data[['latitude', 'longitude', 'created_at']] = data.select_dtypes(include=['object']).apply(lambda x: x.str.strip())

In [148]:
data.latitude.iloc[-1]

''

In [151]:
data.replace(to_replace='', value=np.nan, inplace=True)

In [152]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916,5222,NaN,NaN,2012-04-21 17:39:01
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391,5222,NaN,NaN,2012-04-21 17:37:18
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840,5222,NaN,NaN,2012-04-21 17:42:58


In [154]:
data.dropna(inplace=True)

In [155]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [156]:
data.count()

id            396634
user_id       396634
venue_id      396634
latitude      396634
longitude     396634
created_at    396634
dtype: int64

In [161]:
light_data = data[['latitude', 'longitude']][:100000]

In [162]:
light_data.count()

latitude     100000
longitude    100000
dtype: int64

In [163]:
light_data.head()

,latitude,longitude
1,38.8951118,-77.0363658
3,33.800745,-84.41052
7,45.5234515,-122.6762071
9,40.764462,-111.904565
10,33.4483771,-112.0740373


In [167]:
%%time
clustering = MeanShift(bandwidth=0.1).fit(light_data)

Wall time: 3min 36s


In [185]:
clustering.labels_

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [170]:
clustering.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  38.891565  , -121.2930079 ],
       [  42.5953378 ,  -78.9411461 ],
       [  41.5822716 ,  -85.8344383 ]])

In [193]:
centers = pd.DataFrame(clustering.cluster_centers_)

In [175]:
clusters = pd.Series(clustering.labels_)

In [187]:
clusters.value_counts().head()

0    12471
1     4692
2     3994
4     3526
3     3363
dtype: int64

In [191]:
len(clusters.unique())

3230

In [196]:
centers['labels'] = clusters

In [214]:
centers.head()

,0,1,labels
0,40.717716,-73.991835,5
1,33.449438,-112.002140,7
2,33.446380,-111.901888,30
3,41.878244,-87.629843,65
4,37.688682,-122.409330,1


In [242]:
centers_20 = centers.groupby('labels').filter(lambda x: len(x) > 15)

In [280]:
centers_20.head()

,0,1,labels
0,40.717716,-73.991835,5
1,33.449438,-112.002140,7
2,33.446380,-111.901888,30
4,37.688682,-122.409330,1
5,38.886165,-77.048783,23


In [217]:
offices = pd.DataFrame([[33.751277, -118.188740],
                        [25.867736, -80.324116], 
                        [51.503016, -0.075479], 
                        [52.378894, 4.885084],
                        [39.366487, 117.036146],
                        [-33.868457, 151.205134]])

In [234]:
offices

,0,1
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [240]:
offices_3230 = offices.loc[offices.index.repeat(3230)]

In [258]:
centers_offices = (
    centers_20
    .assign(key=1)
    .merge(offices.assign(key=1), on="key")
    .drop("key", axis=1)
)

In [281]:
centers_offices.head(21)

,0_x,1_x,labels,0_y,1_y,dist
0,40.717716,-73.991835,5,33.751277,-118.188740,44.196905
1,40.717716,-73.991835,5,25.867736,-80.324116,6.332281
2,40.717716,-73.991835,5,51.503016,-0.075479,73.916356
3,40.717716,-73.991835,5,52.378894,4.885084,78.876919
4,40.717716,-73.991835,5,39.366487,117.036146,191.027981
5,40.717716,-73.991835,5,-33.868457,151.205134,225.196969
6,33.449438,-112.002140,7,33.751277,-118.188740,6.186600
7,33.449438,-112.002140,7,25.867736,-80.324116,31.678024
8,33.449438,-112.002140,7,51.503016,-0.075479,111.926661
9,33.449438,-112.002140,7,52.378894,4.885084,116.887224


In [282]:
centers_offices['dist'] = ((centers_offices['0_x'] - centers_offices['0_y'])**2 
+ (centers_offices['1_x'] - centers_offices['1_y'])**2)**0.5

In [283]:
centers_offices.sort_values('dist')

,0_x,1_x,labels,0_y,1_y,dist
1281,52.372964,4.892317,6,52.378894,4.885084,0.009353
204,33.809878,-118.148924,10,33.751277,-118.188740,0.070848
577,25.705350,-80.283429,1,25.867736,-80.324116,0.167406
6137,-33.952263,151.032137,2,-33.868457,151.205134,0.192227
174,33.872986,-118.362091,1,33.751277,-118.188740,0.211811
984,33.972575,-118.168371,1,33.751277,-118.188740,0.222233
3074,51.426763,-0.303732,19,51.503016,-0.075479,0.240653
3861,52.388501,4.633765,11,52.378894,4.885084,0.251502
10994,51.574152,0.183871,6,51.503016,-0.075479,0.268929
8559,52.264400,4.634700,1,52.378894,4.885084,0.275320
